Build a Simple LLM Application using LCEL

LCEL = LangChain Expression Language

In [1]:
### Open AI API Key and Open Source Models -- Llama3, Gemma2, mistral -- Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000263351D7160>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000263351D4C10>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from english to japanese."),
    HumanMessage(content="'Hello, how are you? Thankyou'")
]

result = model.invoke(messages)


In [6]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

output_parser.invoke(result)

"日本語にすると：\n\nこんにちは、お元気ですか？ ありがとうございます。\n\n(Konnichiwa, o-genki desu ka? Arigatou gozaimasu.)\n\n\nHere's a breakdown:\n\n* **こんにちは (Konnichiwa):** Hello\n* **お元気ですか (o-genki desu ka):** How are you? (polite)\n* **ありがとうございます (Arigatou gozaimasu):** Thank you. (formal)\n"

In [7]:
### Using LCEL we can chain the components together
chain = model | output_parser
chain.invoke(messages)

'こんにちは、お元気ですか？ ありがとうございます。\n\n(Konnichiwa, ogenki desu ka? Arigatou gozaimasu.)\n\n\n* こんにちは (Konnichiwa) - Hello\n* お元気ですか (Ogenki desu ka?) - How are you?\n* ありがとうございます (Arigatou gozaimasu) - Thank you \n'

In [10]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate
generic_template = "Translate the following from english to {language}"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user", "{text}"),
    ]
)

result = prompt.invoke({"text": "'I want to eat bread'", "language": "french"})
result.to_messages()

[SystemMessage(content='Translate the following from english to french', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="'I want to eat bread'", additional_kwargs={}, response_metadata={})]

In [11]:
chain = prompt | model | output_parser
chain.invoke({"text": "'I want to eat bread'", "language": "french"})

'Je veux manger du pain. \n'